In [1]:
import sys
import os

# 親ディレクトリの絶対パスを取得
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
# sys.pathに追加
sys.path.insert(0, parent_dir)


In [ ]:
import nest_asyncio
import asyncio
from pathlib import Path
from PIL import Image
import re
import pandas as pd
from io import StringIO

# 既存のイベントループの再利用を許可
nest_asyncio.apply()

# 必要なクラスや関数のインポート
from llm.llmInterface import GlobalConfig, ParallelLLMCaller


In [3]:

print("starting...")
config = GlobalConfig(".\\model_configs.yaml", "..\\.env")
llm = ParallelLLMCaller(config)


starting...


In [ ]:

# テスト用のプロンプト
topics = ["移民", "神", "民主主義"]
prompts = [
    f"{topic} について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に"
    for topic in topics
]

model_name = "gem2f-chat-ja"

print("=== 基本的な並列処理 ===")

# ジュピターノートブックではawaitを直接使う
results = await llm.batch_call(prompts, model_name, max_concurrent=10)

for prompt, result in zip(prompts, results):
    print(f"\n--- {prompt} ---\n{result}")

starting...
=== 基本的な並列処理 ===
[2/3] ✗ 神 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡...
[3/3] ✗ 民主主義 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文...
[1/3] ✗ 移民 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で...

--- 移民 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に ---
LLMResult(prompt='移民 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に', result=None, error='Connection error.', duration=2.3590328999998746, model_name='gem2f-chat-ja', success=False)

--- 神 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に ---
LLMResult(prompt='神 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に', result=None, error='Connection error.', duration=1.3012484999999288, model_name='gem2f-chat-ja', success=False)

--- 民主主義 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に ---
LLMResult(prompt='民主主義 について日本での特別な意味、古代での意味などを答えて。話題に合わせて100文字から500文字で簡潔だが詳細に', result=None, error='Connection error.', duration=1.3012920000037411, model_name='gem2f-chat-ja', success=False)


In [25]:
targetIMGspath = Path("../resources/FileSystem/save/pdf_img/歴史")


pront = """
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こちらは社会の授業プリントです。
丸1番から10番+ほどまでの質問の答えが()で囲まれてかかれています。番号の右側です必ず書きなさい。
またその右側にはメモがあります。
下の方には解説の一部がありますが無視してください。

# output
csvで書きましょう。```csv と``` で囲ったcsv以外を出さないでください。
列は(id,answer,memo)です 
idは英数字のみで連続する形に、(1,2,3)
答えは一つのこさず正確に書き写しなさい。なにも書かれていないのはだめです。
memoはほとんどについています。ついていなければ""を書きましょう
ついている場合は簡潔な文章になるように元のものを少し変えて書きます。平仮名でなく漢字で置き換えるなど

例:
```csv
id,answer,memo
1,鎌倉時代,武士の政権が始まった時代
2,徳川家康,江戸幕府を開いた人物
```
"""

images = []

In [26]:

image = Image.open("../resources/FileSystem/save/pdf_img/歴史/page_001.jpeg")
responce = await llm.single_call("gem2f-ocr-ja",pront,image)

print(responce.error)
print(responce.result)

None
```csv
id,answer,memo
1,免罪符,金さえ出せば罪が許され天国へ行ける
2,ルター,カトリックの神父
3,カルヴァン,スイスのジュネーブで独自の教会を設立
4,ヘンリ4世,離婚問題からカトリックを離脱、イギリス国教会を設立
5,カール5世,神聖ローマ帝国皇帝、宗教改革を弾圧
6,三十年戦争,ドイツ1618年~1648年
7,ウェストファリア条約,で戦争が終結
8,絶対王政,""
9,エリザベス1世,イングランドの女王
10,無敵艦隊,スペインが派遣したがイングランド海軍により壊滅
11,チャールズ1世,""
12,ピューリタン革命,1641年
13,名誉革命,フランスとの友好、カトリック保護でジェームズ2世を追放
```


In [27]:
# `````` に囲まれた部分を抽出
pattern = r'```csv\s*(.*?)\s*```'
match = re.search(pattern, str(responce.result), re.DOTALL)
csv_text = match.group(1).strip() if match else ""

# DataFrameへ変換
df = pd.read_csv(StringIO(csv_text))

print(df)

    id      answer                         memo
0    1         免罪符            金さえ出せば罪が許され天国へ行ける
1    2         ルター                     カトリックの神父
2    3       カルヴァン           スイスのジュネーブで独自の教会を設立
3    4       ヘンリ4世    離婚問題からカトリックを離脱、イギリス国教会を設立
4    5       カール5世            神聖ローマ帝国皇帝、宗教改革を弾圧
5    6       三十年戦争               ドイツ1618年~1648年
6    7  ウェストファリア条約                       で戦争が終結
7    8        絶対王政                          NaN
8    9     エリザベス1世                    イングランドの女王
9   10        無敵艦隊      スペインが派遣したがイングランド海軍により壊滅
10  11     チャールズ1世                          NaN
11  12    ピューリタン革命                        1641年
12  13        名誉革命  フランスとの友好、カトリック保護でジェームズ2世を追放


In [28]:
# targetIMGspath配下の画像ファイルをループ
for img_path in targetIMGspath.iterdir():
    if img_path.is_file():
        # 画像ファイルを開く
        img = Image.open(img_path)
        images.append(img)

responces = await llm.batch_call_listIMG(pront,images,"gem2f-ocr-ja")


[5/6] ✓ 
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こ...
[6/6] ✓ 
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こ...
[2/6] ✓ 
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こ...
[3/6] ✓ 
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こ...
[1/6] ✓ 
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こ...
[4/6] ✓ 
# task 
日本語でOCRしてください。
答えとメモを書き起こしましょう。

# info
こ...


In [30]:
datas = []

for responce in responces:
    # `````` に囲まれた部分を抽出
    pattern = r'```csv\s*(.*?)\s*```'
    match = re.search(pattern, str(responce.result), re.DOTALL)
    csv_text = match.group(1).strip() if match else ""

    # DataFrameへ変換
    df = pd.read_csv(StringIO(csv_text))

    datas.append(df)

In [33]:
# 方法2: より明示的な書き方
dfs_with_index = []
for i, df in enumerate(datas):
    df_copy = df.copy()
    df_copy['list_index'] = i
    dfs_with_index.append(df_copy)

result_df = pd.concat(dfs_with_index, ignore_index=True)
result_df = result_df[['list_index', 'id', 'answer', 'memo']]

print("結合後のDataFrame:")
print(result_df.head())
print(f"\n形状: {result_df.shape}")
print(f"カラム: {list(result_df.columns)}")

# 各list_indexごとの件数確認
print("\n各リストインデックスごとの件数:")
print(result_df['list_index'].value_counts().sort_index())

結合後のDataFrame:
   list_index  id answer                                memo
0           0   1    免罪符                   金さえ出せば罪が許され天国へ行ける
1           0   2    ルター                            カトリックの神父
2           0   3  カルヴァン                  スイスのジュネーブで独自の教会を設立
3           0   4  ヘンリ4世  離婚問題からカトリックを離脱、イギリス国教会を設立、イングランド宗教
4           0   5  カール5世                     神聖ローマ帝国皇帝、改革を弾圧

形状: (61, 4)
カラム: ['list_index', 'id', 'answer', 'memo']

各リストインデックスごとの件数:
list_index
0    13
1    13
2    10
3     7
4     8
5    10
Name: count, dtype: int64


In [34]:
result_df.head(-1)

,list_index,id,answer,memo
0,0,1,免罪符,金さえ出せば罪が許され天国へ行ける
1,0,2,ルター,カトリックの神父
2,0,3,カルヴァン,スイスのジュネーブで独自の教会を設立
3,0,4,ヘンリ4世,離婚問題からカトリックを離脱、イギリス国教会を設立、イングランド宗教
4,0,5,カール5世,神聖ローマ帝国皇帝、改革を弾圧
5,0,6,三十年戦争,ドイツ1618年～1648年
6,0,7,ウェストファリア条約,で戦争が終結
7,0,8,絶対王政,NaN
8,0,9,エリザベス1世,イングランドの女王
9,0,10,無敵艦隊,スペインが派遣したがイングランド海軍により壊滅
